In [3]:
class ReplayMemoryBuffer(object):
    
    def __init__(self, source, low_water_mark, high_water_mark = None):
        if high_water_mark is None:
            high_water_mark = int(1.2 * low_water_mark)
        self.LowWaterMark = low_water_mark
        self.HighWaterMark = high_water_mark
        self.Memory = []
        self.Source = source
        
    def fill(self, requested_size):
        while len(self.Memory) < max(requested_size, self.HighWaterMark):
            rows = self.Source.generate()
            self.Memory += rows

    def sample(self, n):
        if len(self.Memory) < max(n, self.LowWaterMark):
            self.fill(n)
            self.shuffle()
        assert len(self.Memory) >= n
        data = self.Memory[:n]
        self.Memory = self.Memory[n:]
        return data

    NParts = 4
    
    def shuffle_once(self):
        n = len(self.Memory)
        done = False
        while not done:
            inx = np.random.randint(n, self.NParts-1).sort()
            done = inx[0] != 0 and inx[-1] != n-1 and np.all(inx[1:]!=inx[:-1])
        i = 0
        parts = []
        for j in inx:
            parts.append(self.Memory[i:j])
            i = j
        parts.append(self.Memory[i:])
        parts_order = np.arange(self.NParts)
        np.shuffle(parts_order)
        parts = [parts[j] for j in parts_order]
        self.Memory = sum(parts, [])

    NShuffle = 3
        
    def shuffle(self, n = NShuffle):
        for _ in xrange(n):
            self.shuffle_once()
        
        

In [2]:
class generator(object):
    

[1, 2, 3, 4, 5]